# Chain‑of‑Thought & Self‑Consistency (Few‑Shot Prompting)

In [ ]:
!pip -q install -U transformers accelerate


In [ ]:
import torch, random
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

base = "google/flan-t5-small"
tok = AutoTokenizer.from_pretrained(base)
model = AutoModelForSeq2SeqLM.from_pretrained(base).to("cuda" if torch.cuda.is_available() else "cpu")

examples = [
    ("If there are 3 boxes with 2 apples each, how many apples total?", "Let's think step by step. There are 3 boxes and each has 2 apples, so 3*2 = 6. The answer is 6."),
    ("Tom has 5 red balloons and buys 4 more. How many now?", "Let's think step by step. He had 5 and buys 4 more, so 5+4=9. The answer is 9."),
]

def cot_prompt(q):
    shots = "\n\n".join([f"Q: {a}\nA: {b}" for a,b in examples])
    return shots + f"\n\nQ: {q}\nA: Let's think step by step."

question = "A farmer has 8 cows and buys 7 more, then sells 5. How many cows remain?"
inp = tok(cot_prompt(question), return_tensors="pt").to(model.device)
outs = [tok.decode(model.generate(**inp, max_new_tokens=64, do_sample=True, top_p=0.9, temperature=0.7)[0], skip_special_tokens=True) for _ in range(5)]
outs


In [ ]:
# Self-consistency: take majority answer
import re
def extract_answer(s):
    m = re.findall(r"(?:answer is|=)\s*(\d+)", s)
    return m[-1] if m else None

answers = [extract_answer(o) for o in outs]
print("Candidates:", answers)
from collections import Counter
cnt = Counter([a for a in answers if a is not None])
print("Majority vote:", cnt.most_common(1)[0] if cnt else "No consensus")


> Caveat: CoT increases tokens and often verbosity; combine with robust evaluation and, when possible, process supervision.